### 背景
现实世界中的时间序列预测具有很大的挑战性，其原因不仅限于问题特征，例如具有多个输入变量，需要预测多个时间步长以及需要对多个物理站点执行相同类型的预测。

EMC数据科学全球Hackathon数据集，或简称为“空气质量预测”数据集，描述了多个站点的天气状况，并要求对随后三天的空气质量测量值进行预测。

机器学习算法可以应用于时间序列预测问题，并提供诸如处理带有复杂杂讯的多个输入变量的功能。

### 问题描述
空气质量预测数据集描述了多个地点的天气状况，并要求对随后三天的空气质量测量值进行预测。

具体来说，每小时为多个站点提供为期八天的天气观测信息，例如温度，压力，风速和风向。目的是预测未来3天在多个地点的空气质量测量结果。预测的交货时间不连续；相反，必须在72小时的预测期内预测特定的交货时间。他们是：
+1, +2, +3, +4, +5, +10, +17, +24, +48, +72
此外，该数据集被分为不相交但连续的数据块，其中八天的数据紧随其后的三天需要预测。

并非所有观察值在所有站点或块均可用，并且并非所有输出变量在所有站点或块均可用。在丢失的数据中，有很大一部分必须加以解决。

该数据集被用作2012年Kaggle网站上的短期机器学习竞赛（或hackathon）的基础。

比赛提交的内容是根据参与者保留的真实观察结果进行评估的，并使用平均绝对误差（MAE）进行评分。

### 模型评估
在评估朴素的预测方法之前，我们必须开发一种测试工具。

这至少包括如何准备数据以及如何评估预测。

### 加载数据集
第一步是下载数据集并将其加载到内存中。

我们的重点是包含训练数据集的“ TrainingData.csv ”文件，特别是大块中的数据，其中每个大块是连续八天的观测值和目标变量。

我们可以使用Pandas read_csv（）函数将数据文件加载到内存中，并在第0行指定标题行。

我们可以按“ chunkID”变量（列索引1）对数据进行分组。

下面定义了一个名为to_chunks（）的函数，该函数接受已加载数据的NumPy数组，并将chunk_id的字典返回给该块的行。

In [1]:
# load data and split into chunks
from numpy import unique
from pandas import read_csv
 
# split the dataset by 'chunkID', return a dict of id to rows
def to_chunks(values, chunk_ix=1):
	chunks = dict()
	# get the unique chunk ids
	chunk_ids = unique(values[:, chunk_ix])
	# group rows by chunk id
	for chunk_id in chunk_ids:
		selection = values[:, chunk_ix] == chunk_id
		chunks[chunk_id] = values[selection, :]
	return chunks
 
# load dataset
dataset = read_csv('../AirQualityPrediction/TrainingData.csv', header=0)
# group data by chunks
values = dataset.values
chunks = to_chunks(values)
print('Total Chunks: %d' % len(chunks))

Total Chunks: 208


### 资料准备
现在我们知道了如何加载数据并将其拆分为大块，我们可以将其分为训练数据集和测试数据集。

尽管每个块内的实际观察次数可能相差很大，但是每个块的时间间隔为每小时观察八天。

我们可以将每个块分为观察的前五天进行训练，最后三天进行测试。

每个观察值都有一个称为“ position_within_chunk ” 的行，范围从1到192（8天* 24小时）不等。因此，我们可以将此列中具有小于或等于120（5 * 24）的值的所有行作为训练数据，并将大于120的任何值作为测试数据。

此外，在训练或测试拆分中没有任何观察结果的任何块都可以丢弃为不可行。

当使用朴素模型时，我们只对目标变量感兴趣，而对输入的气象变量都不感兴趣。因此，我们可以删除输入数据，并使训练和测试数据仅包含每个块的39个目标变量，以及在块内和观察小时内的位置。

下面的split_train_test（）函数实现了此行为；给定一个大块字典，它将把每个大块分解为训练和测试大块数据列表。

我们不需要整个测试数据集；取而代之的是，我们只需要在三天内的特定提前期进行观察，尤其是提前期：+1, +2, +3, +4, +5, +10, +17, +24, +48, +72

首先，我们可以将这些提前期放入函数中以便于参考：

def get_lead_times():   
	return [1, 2 ,3, 4, 5, 10, 17, 24, 48, 72]

接下来，我们可以将测试数据集缩减为仅在首选交货时间处的数据。

我们可以通过查看“ position_within_chunk ”列并使用提前期作为距训练数据集末尾的偏移量（例如120 + 1、120 +2等）来实现。

如果在测试集中找到匹配的行，则将其保存，否则将生成一行NaN观测值。

下面的函数to_forecasts（）实现了此功能，并返回一个NumPy数组，其中每个块的每个预测提前期都有一行。

我们可以将所有这些结合在一起，并将数据集分为训练集和测试集，然后将结果保存到新文件中。

In [9]:
# split data into train and test sets
from numpy import unique
from numpy import nan
from numpy import array
from numpy import savetxt
from pandas import read_csv
 
# split the dataset by 'chunkID', return a dict of id to rows
def to_chunks(values, chunk_ix=1):
	chunks = dict()
	# get the unique chunk ids
	chunk_ids = unique(values[:, chunk_ix])
	# group rows by chunk id
	for chunk_id in chunk_ids:
		selection = values[:, chunk_ix] == chunk_id
		chunks[chunk_id] = values[selection, :]
	return chunks
 
# split each chunk into train/test sets
def split_train_test(chunks, row_in_chunk_ix=2):
	train, test = list(), list()
	# first 5 days of hourly observations for train
	cut_point = 5 * 24
	# enumerate chunks
	for k,rows in chunks.items():
		# split chunk rows by 'position_within_chunk'
		train_rows = rows[rows[:,row_in_chunk_ix] <= cut_point, :]
		test_rows = rows[rows[:,row_in_chunk_ix] > cut_point, :]
		if len(train_rows) == 0 or len(test_rows) == 0:
			print('>dropping chunk=%d: train=%s, test=%s' % (k, train_rows.shape, test_rows.shape))
			continue
		# store with chunk id, position in chunk, hour and all targets
		indices = [1,2,5] + [x for x in range(56,train_rows.shape[1])]
		train.append(train_rows[:, indices])
		test.append(test_rows[:, indices])
	return train, test
 
# return a list of relative forecast lead times
def get_lead_times():
	return [1, 2 ,3, 4, 5, 10, 17, 24, 48, 72]
 
# convert the rows in a test chunk to forecasts
def to_forecasts(test_chunks, row_in_chunk_ix=1):
	# get lead times
	lead_times = get_lead_times()
	# first 5 days of hourly observations for train
	cut_point = 5 * 24
	forecasts = list()
	# enumerate each chunk
	for rows in test_chunks:
		chunk_id = rows[0, 0]
		# enumerate each lead time
		for tau in lead_times:
			# determine the row in chunk we want for the lead time
			offset = cut_point + tau
			# retrieve data for the lead time using row number in chunk
			row_for_tau = rows[rows[:,row_in_chunk_ix]==offset, :]
			# check if we have data
			if len(row_for_tau) == 0:
				# create a mock row [chunk, position, hour] + [nan...]
				row = [chunk_id, offset, nan] + [nan for _ in range(39)]
				forecasts.append(row)
			else:
				# store the forecast row
				forecasts.append(row_for_tau[0])
	return array(forecasts)
 
# load dataset
dataset = read_csv('../AirQualityPrediction/TrainingData.csv', header=0)
# group data by chunks
values = dataset.values
chunks = to_chunks(values)
# split into train/test
train, test = split_train_test(chunks)
# flatten training chunks to rows
train_rows = array([row for rows in train for row in rows])
# print(train_rows.shape)
print('Train Rows: %s' % str(train_rows.shape))
# reduce train to forecast lead times only
test_rows = to_forecasts(test)
print('Test Rows: %s' % str(test_rows.shape))
# save datasets
savetxt('naive_train.csv', train_rows, delimiter=',')
savetxt('naive_test.csv', test_rows, delimiter=',')

>dropping chunk=69: train=(0, 95), test=(28, 95)
Train Rows: (23514, 42)
Test Rows: (2070, 42)


运行示例第一条注释，因为数据不足，从数据集中删除了块69。

然后我们可以看到，每个训练集和测试集中都有42列，其中一列表示块ID，块内的位置，一天中的小时以及39个训练变量。

我们还可以看到测试数据集的版本大大缩小，仅在预测提前期有行。

新的训练和测试数据集分别保存在“ naive_train.csv ”和“ naive_test.csv ”文件中。

### 预测评估
做出预测后，需要对其进行评估。

评估预测时使用更简单的格式很有帮助。例如，我们将使用[chunks] [variables] [time]的三维结构，其中variable是目标变量号（从0到38），而time是提前期索引从0到9。

预计模型将以这种格式进行预测。

我们还可以重组测试数据集以使该数据集进行比较。下面的prepare_test_forecasts（）函数实现了这一点。

In [11]:
# convert the test dataset in chunks to [chunk][variable][time] format
def prepare_test_forecasts(test_chunks):
	predictions = list()
	# enumerate chunks to forecast
	for rows in test_chunks:
		# enumerate targets for chunk
		chunk_predictions = list()
		for j in range(3, rows.shape[1]):
			yhat = rows[:, j]
			chunk_predictions.append(yhat)
		chunk_predictions = array(chunk_predictions)
		predictions.append(chunk_predictions)
	return array(predictions)


我们将使用平均绝对误差或MAE评估模型。这是比赛中使用的指标，考虑到目标变量的非高斯分布，这是明智的选择。

如果提前期在测试集中不包含任何数据（例如NaN），则不会为该预测计算错误。如果提前期在测试集中确实有数据，但预测中没有数据，则观测值的全部幅度将被视为误差。最后，如果测试集具有观测值并做出了预测，则绝对差将记录为误差。

该calculate_error（）函数实现这些规则，并返回错误对于一个给定的预测。

In [10]:
# calculate the error between an actual and predicted value
def calculate_error(actual, predicted):
	# give the full actual value if predicted is nan
	if isnan(predicted):
		return abs(actual)
	# calculate abs difference
	return abs(actual - predicted)

将所有块和所有提前期的错误求和，然后平均。

将计算总体MAE，但我们还将为每个预测的提前期计算MAE。通常，这可以帮助选择模型，因为某些模型在不同的交货时间可能会有不同的表现。

下面的evaluate_forecasts（）函数实现了此功能，以[chunk] [variable] [time]格式为提供的预测和期望值计算了MAE和提前期MAE 。

In [12]:
# evaluate a forecast in the format [chunk][variable][time]
def evaluate_forecasts(predictions, testset):
	lead_times = get_lead_times()
	total_mae, times_mae = 0.0, [0.0 for _ in range(len(lead_times))]
	total_c, times_c = 0, [0 for _ in range(len(lead_times))]
	# enumerate test chunks
	for i in range(len(test_chunks)):
		# convert to forecasts
		actual = testset[i]
		predicted = predictions[i]
		# enumerate target variables
		for j in range(predicted.shape[0]):
			# enumerate lead times
			for k in range(len(lead_times)):
				# skip if actual in nan
				if isnan(actual[j, k]):
					continue
				# calculate error
				error = calculate_error(actual[j, k], predicted[j, k])
				# update statistics
				total_mae += error
				times_mae[k] += error
				total_c += 1
				times_c[k] += 1
	# normalize summed absolute errors
	total_mae /= total_c
	times_mae = [times_mae[i]/times_c[i] for i in range(len(times_mae))]
	return total_mae, times_mae

一旦我们对模型进行了评估，就可以提出它。

下面的summary_error（）函数首先打印模型性能的单行摘要，然后创建每个预测提前期的MAE图。

In [13]:
# summarize scores
def summarize_error(name, total_mae, times_mae):
	# print summary
	lead_times = get_lead_times()
	formatted = ['+%d %.3f' % (lead_times[i], times_mae[i]) for i in range(len(lead_times))]
	s_scores = ', '.join(formatted)
	print('%s: [%.3f MAE] %s' % (name, total_mae, s_scores))
	# plot summary
	pyplot.plot([str(x) for x in lead_times], times_mae, marker='.')
	pyplot.show()

机器学习建模

可以使用机器学习对问题进行建模。

大多数机器学习模型并不直接支持随着时间的推移观察的概念。取而代之的是，必须将滞后观测值视为输入特征，以便进行预测。

这是机器学习算法用于时间序列预测的好处。特别是，它们能够支持大量的输入功能。这些可能是一个或多个输入时间序列的滞后观察。

与传统方法相比，用于时间序列预测的机器学习算法的其他一般优势包括：

在变量之间的关系中支持噪声特征和噪声的能力。
处理无关功能的能力。
支持变量之间的复杂关系的能力。
该数据集的挑战是需要进行多步预测。机器学习方法可用于进行多步预测的方法主要有两种：他们是：

直接。开发了一个单独的模型来预测每个预测的提前期。
递归的。开发了一个模型来进行一步预测，然后递归使用该模型，其中将先前的预测用作输入以预测后续的交货时间。
当预测一个短的连续交付周期时，递归方法可能有意义，而当预测不连续的交付周期时，直接方法可能更有意义。鉴于我们有兴趣在三天内预测10个连续和不连续的交货时间，因此直接方法可能更适合于空气污染预测问题。

数据集有39个目标变量，并且每个预测提前期，我们为每个目标变量开发一个模型。这意味着我们需要（39 * 10）390个机器学习模型。

使用机器学习算法进行时间序列预测的关键是选择输入数据。我们可以考虑三个主要数据源，这些数据源可用作输入并映射到目标变量的每个预测提前期。他们是：

单变量数据，例如来自预测目标变量的滞后观测值。
多元数据，例如来自其他变量（天气和目标）的滞后观测值。
元数据，例如有关预测日期或时间的数据。
可以从所有块中提取数据，从而提供丰富的数据集，以学习从输入到目标预测提前期的映射。

39个目标变量实际上由14个站点的12个变量组成。

由于提供数据的方式，默认的建模方法是将每个可变位置都视为独立位置。可能可以按变量折叠数据，并且可以跨多个站点对变量使用相同的模型。

有些变量被故意贴错标签（例如，不同数据使用具有相同标识符的变量）。但是，也许可以识别这些标签错误的变量并将其从多站点模型中排除。

### 机器学习数据准备
在探索该数据集的机器学习模型之前，我们必须以适合模型的方式准备数据。

这需要两个数据准备步骤：

#### 处理丢失的数据
准备输入输出模式。
现在，我们将重点关注39个目标变量，而忽略气象和元数据。

处理丢失的数据
每小时对39个目标变量进行5天或更少的观察。

许多块没有全部五天的数据，并且所有块都不具有所有39个目标变量的数据。

在块中没有目标变量数据的情况下，不需要预测。

在那些大块确实有一些目标变量数据但并非全部五天有效的情况下，序列中会有缺口。在一天的观察中，这些间隔可能需要几个小时到几天，有时甚至更长。

解决这些差距的三种候选策略如下：

忽略差距。
使用无间隙的数据。
填补空白。
我们可以忽略差距。这样做的问题是，将数据分为输入和输出时，数据将不是连续的。训练模型时，输入的数据将不一致，但可能意味着最后n个小时的数据，或者数据分散在过去n天内。这种不一致将使学习从输入到输出的映射变得非常嘈杂，对于模型而言，可能比需要的困难得多。

我们只能使用没有差距的数据。这是一个不错的选择。风险在于我们可能没有太多或足够的数据来拟合模型。

最后，我们可以填补空白。这称为数据插补，可以使用许多策略来填补空白。三种可能效果良好的方法包括：

持续保留最后一个观测值（线性）。
使用大块中一天中小时的中位数。
在一天中的各个小时中使用大块中位数。
在本教程中，我们将使用后一种方法并通过使用一天中各个块的中位数来填补空白。经过一点测试，这种方法似乎可以产生更多的训练样本和更好的模型性能。

对于给定的变量，可能缺少由缺少的行定义的观察值。具体地，每个观测具有“ position_within_chunk ”。我们预计，在训练数据集每个块有120个观察，与“ positions_within_chunk ”从1到120包含边界。

因此，我们可以为每个变量创建一个由120个NaN值组成的数组，使用“ position_within_chunk ”值标记该块中的所有观测值，剩下的任何内容都将标记为  NaN。然后，我们可以绘制每个变量并寻找差距。

下面的variable_to_series（）函数将获取块的行和目标变量的给定列索引，并将为该变量返回一系列120个时间步长，所有可用数据均标有块中的值。

In [14]:
# layout a variable with breaks in the data for missing positions
def variable_to_series(chunk_train, col_ix, n_steps=5*24):
	# lay out whole series
	data = [nan for _ in range(n_steps)]
	# mark all available data
	for i in range(len(chunk_train)):
		# get position in chunk
		position = int(chunk_train[i, 1] - 1)
		# store data
		data[position] = chunk_train[i, col_ix]
	return data

我们需要为每个块计算一天中小时的并行序列，我们可以使用该序列来为块中的每个变量估算特定于小时的数据。

给定一系列一天中的部分时段，下面的interpolate_hours（）函数将填充一天中缺少的时段。它通过找到第一个标记的小时，然后向前计数，填写一天中的小时，然后向后执行相同的操作来做到这一点。

In [15]:
# interpolate series of hours (in place) in 24 hour time
def interpolate_hours(hours):
	# find the first hour
	ix = -1
	for i in range(len(hours)):
		if not isnan(hours[i]):
			ix = i
			break
	# fill-forward
	hour = hours[ix]
	for i in range(ix+1, len(hours)):
		# increment hour
		hour += 1
		# check for a fill
		if isnan(hours[i]):
			hours[i] = hour % 24
	# fill-backward
	hour = hours[ix]
	for i in range(ix-1, -1, -1):
		# decrement hour
		hour -= 1
		# check for a fill
		if isnan(hours[i]):
			hours[i] = hour % 24

我们可以调用相同的variable_to_series（）函数（上面）来创建缺少值的小时系列（列索引2），然后调用interpolate_hours（）来填补空白。

然后，我们可以将小时数传递给任何可能使用它的归因函数。

现在，我们可以尝试使用同一小时内相同系列中的值来填充大块中的缺失值。具体来说，我们将在序列中找到所有具有相同小时数的行，并计算中值。

下面的impute_missing（）会接受一个块中的所有行，为该块中一天中的小时准备好的序列，以及一个缺少变量值和变量的列索引的序列。

它首先检查该序列是否全部缺少数据，如果是这种情况，则由于无法进行估算，因此会立即返回。然后，它枚举序列的时间步长，当它检测到没有数据的时间步长时，它将收集同一小时内数据的序列中的所有行，并计算中位数。

In [16]:
# impute missing data
def impute_missing(train_chunks, rows, hours, series, col_ix):
	# impute missing using the median value for hour in all series
	imputed = list()
	for i in range(len(series)):
		if isnan(series[i]):
			# collect all rows across all chunks for the hour
			all_rows = list()
			for rows in train_chunks:
				[all_rows.append(row) for row in rows[rows[:,2]==hours[i]]]
			# calculate the central tendency for target
			all_rows = array(all_rows)
			# fill with median value
			value = nanmedian(all_rows[:, col_ix])
			if isnan(value):
				value = 0.0
			imputed.append(value)
		else:
			imputed.append(series[i])
	return imputed


### 监督学习的数据表征方式
我们需要将每个目标变量的序列转换为具有输入和输出的行，以便适合监督的机器学习算法。

具体来说，我们有一系列，例如：

[1, 2, 3, 4, 5, 6, 7, 8, 9]   
当使用2个滞后变量预测+1的提前期时，我们将序列分为输入（X）和输出（y）模式，如下所示：

X,			y   
1, 2,		3   
2, 3,		4   
3, 4,		5   
4, 5,		6   
5, 6,		7   
6, 7,		8   
7, 8,		9   

这首先要求我们选择一些滞后观测值作为输入。没有正确的答案; 相反，最好测试不同的数字并查看有效的方法。

然后，我们必须针对10个预测提前期中的每一个，将系列划分为有监督的学习格式。例如，用2个滞后观测值预测+24可能看起来像：

X,			y   
1, 2,		24   

然后，对39个目标变量中的每一个重复此过程。

然后可以跨每个块聚合为每个目标变量的每个提前期准备的模式，以提供模型的训练数据集。

我们还必须准备一个测试数据集。也就是说，为每个块的每个目标变量输入数据（X），以便我们可以将其用作输入来预测测试数据集中的提前期。如果我们选择2的滞后，则测试数据集将由每个块的每个目标变量的最后两个观察值组成。非常简单。

我们可以通过定义一个函数来开始，该函数将为给定的完整（插补）系列创建输入-输出模式。

下面的supervised_for_lead_time（）函数将采用一个序列，将许多滞后观察值用作输入，并使用预测提前期进行预测，然后将返回从该序列中得出的输入/输出行的列表。

In [17]:
# created input/output patterns from a sequence
def supervised_for_lead_time(series, n_lag, lead_time):
	samples = list()
	# enumerate observations and create input/output patterns
	for i in range(n_lag, len(series)):
		end_ix = i + (lead_time - 1)
		# check if can create a pattern
		if end_ix >= len(series):
			break
		# retrieve input and output
		start_ix = i - n_lag
		row = series[start_ix:i] + [series[end_ix]]
		samples.append(row)
	return samples

了解这一部分很重要。

我们可以测试此功能，并在一个小的测试数据集上探索不同数量的滞后变量和预测提前期。

以下是一个完整的示例，该示例生成一系列20个整数，并创建一个具有两个输入滞后的序列，并预测+6提前期。

In [18]:
# test supervised to input/output patterns
from numpy import array

# created input/output patterns from a sequence
def supervised_for_lead_time(series, n_lag, lead_time):
	data = list()
	# enumerate observations and create input/output patterns
	for i in range(n_lag, len(series)):
		end_ix = i + (lead_time - 1)
		# check if can create a pattern
		if end_ix >= len(series):
			break
		# retrieve input and output
		start_ix = i - n_lag
		row = series[start_ix:i] + [series[end_ix]]
		data.append(row)
	return array(data)

# define test dataset
data = [x for x in range(20)]
# convert to supervised format
result = supervised_for_lead_time(data, 2, 6)
# display result
print(result)

[[ 0  1  7]
 [ 1  2  8]
 [ 2  3  9]
 [ 3  4 10]
 [ 4  5 11]
 [ 5  6 12]
 [ 6  7 13]
 [ 7  8 14]
 [ 8  9 15]
 [ 9 10 16]
 [10 11 17]
 [11 12 18]
 [12 13 19]]


运行示例将打印出结果模式，这些模式显示出滞后观测值及其相关的预测提前期。

对此示例进行实验，以使其适应这种数据转换，因为它是使用机器学习算法对时间序列进行建模的关键。

现在，我们可以为给定目标变量系列的每个预测提前期调用supervised_for_lead_time（）。

下面的target_to_supervised（）函数实现了这一点。首先，使用上一节中开发的函数将目标变量转换为序列并进行插补。然后针对每个目标提前期创建训练样本。还创建了目标变量的测试样本。

然后，针对该目标变量返回针对每个预测提前期的训练数据和测试输入数据。

In [19]:
# create supervised learning data for each lead time for this target
def target_to_supervised(chunks, rows, hours, col_ix, n_lag):
	train_lead_times = list()
	# get series
	series = variable_to_series(rows, col_ix)
	if not has_data(series):
		return None, [nan for _ in range(n_lag)]
	# impute
	imputed = impute_missing(chunks, rows, hours, series, col_ix)
	# prepare test sample for chunk-variable
	test_sample = array(imputed[-n_lag:])
	# enumerate lead times
	lead_times = get_lead_times()
	for lead_time in lead_times:
		# make input/output data from series
		train_samples = supervised_for_lead_time(imputed, n_lag, lead_time)
		train_lead_times.append(train_samples)
	return train_lead_times, test_sample

我们有碎片；我们现在需要定义函数来驱动数据准备过程。

此功能建立训练和测试数据集。

该方法是枚举每个目标变量，并从所有块中收集每个提前期的训练数据。同时，在对测试数据集进行预测时，我们收集所需的样本作为输入。

结果是训练数据集，其维度为[var] [提前期] [sample]，其中最终维度为目标变量的预测提前期的培训样本行。该函数还返回维度为[chunk] [var] [sample]的测试数据集，其中最终维度是用于对块的目标变量进行预测的输入数据。

下面的data_prep（）函数实现了此行为，并采用块格式的数据和指定数量的滞后观察值用作输入。

In [20]:
# prepare training [var][lead time][sample] and test [chunk][var][sample]
def data_prep(chunks, n_lag, n_vars=39):
	lead_times = get_lead_times()
	train_data = [[list() for _ in range(len(lead_times))] for _ in range(n_vars)]
	test_data = [[list() for _ in range(n_vars)] for _ in range(len(chunks))]
	# enumerate targets for chunk
	for var in range(n_vars):
		# convert target number into column number
		col_ix = 3 + var
		# enumerate chunks to forecast
		for c_id in range(len(chunks)):
			rows = chunks[c_id]
			# prepare sequence of hours for the chunk
			hours = variable_to_series(rows, 2)
			# interpolate hours
			interpolate_hours(hours)
			# check for no data
			if not has_data(rows[:, col_ix]):
				continue
			# convert series into training data for each lead time
			train, test_sample = target_to_supervised(chunks, rows, hours, col_ix, n_lag)
			# store test sample for this var-chunk
			test_data[c_id][var] = test_sample
			if train is not None:
				# store samples per lead time
				for lead_time in range(len(lead_times)):
					# add all rows to the existing list of rows
					train_data[var][lead_time].extend(train[lead_time])
		# convert all rows for each var-lead time to a numpy array
		for lead_time in range(len(lead_times)):
			train_data[var][lead_time] = array(train_data[var][lead_time])
	return array(train_data), array(test_data)

### 完整的例子
我们可以将所有内容结合在一起，并使用受监督的学习格式为机器学习算法准备训练和测试数据集。

在预测每个预测的提前期时，我们将使用之前的12个小时的滞后观测作为输入。

然后将生成的训练和测试数据集保存为二进制NumPy数组。

下面列出了完整的示例。

In [21]:
# prepare data
from numpy import loadtxt
from numpy import nan
from numpy import isnan
from numpy import count_nonzero
from numpy import unique
from numpy import array
from numpy import nanmedian
from numpy import save

# split the dataset by 'chunkID', return a list of chunks
def to_chunks(values, chunk_ix=0):
	chunks = list()
	# get the unique chunk ids
	chunk_ids = unique(values[:, chunk_ix])
	# group rows by chunk id
	for chunk_id in chunk_ids:
		selection = values[:, chunk_ix] == chunk_id
		chunks.append(values[selection, :])
	return chunks

# return a list of relative forecast lead times
def get_lead_times():
	return [1, 2, 3, 4, 5, 10, 17, 24, 48, 72]

# interpolate series of hours (in place) in 24 hour time
def interpolate_hours(hours):
	# find the first hour
	ix = -1
	for i in range(len(hours)):
		if not isnan(hours[i]):
			ix = i
			break
	# fill-forward
	hour = hours[ix]
	for i in range(ix+1, len(hours)):
		# increment hour
		hour += 1
		# check for a fill
		if isnan(hours[i]):
			hours[i] = hour % 24
	# fill-backward
	hour = hours[ix]
	for i in range(ix-1, -1, -1):
		# decrement hour
		hour -= 1
		# check for a fill
		if isnan(hours[i]):
			hours[i] = hour % 24

# return true if the array has any non-nan values
def has_data(data):
	return count_nonzero(isnan(data)) < len(data)

# impute missing data
def impute_missing(train_chunks, rows, hours, series, col_ix):
	# impute missing using the median value for hour in all series
	imputed = list()
	for i in range(len(series)):
		if isnan(series[i]):
			# collect all rows across all chunks for the hour
			all_rows = list()
			for rows in train_chunks:
				[all_rows.append(row) for row in rows[rows[:,2]==hours[i]]]
			# calculate the central tendency for target
			all_rows = array(all_rows)
			# fill with median value
			value = nanmedian(all_rows[:, col_ix])
			if isnan(value):
				value = 0.0
			imputed.append(value)
		else:
			imputed.append(series[i])
	return imputed

# layout a variable with breaks in the data for missing positions
def variable_to_series(chunk_train, col_ix, n_steps=5*24):
	# lay out whole series
	data = [nan for _ in range(n_steps)]
	# mark all available data
	for i in range(len(chunk_train)):
		# get position in chunk
		position = int(chunk_train[i, 1] - 1)
		# store data
		data[position] = chunk_train[i, col_ix]
	return data

# created input/output patterns from a sequence
def supervised_for_lead_time(series, n_lag, lead_time):
	samples = list()
	# enumerate observations and create input/output patterns
	for i in range(n_lag, len(series)):
		end_ix = i + (lead_time - 1)
		# check if can create a pattern
		if end_ix >= len(series):
			break
		# retrieve input and output
		start_ix = i - n_lag
		row = series[start_ix:i] + [series[end_ix]]
		samples.append(row)
	return samples

# create supervised learning data for each lead time for this target
def target_to_supervised(chunks, rows, hours, col_ix, n_lag):
	train_lead_times = list()
	# get series
	series = variable_to_series(rows, col_ix)
	if not has_data(series):
		return None, [nan for _ in range(n_lag)]
	# impute
	imputed = impute_missing(chunks, rows, hours, series, col_ix)
	# prepare test sample for chunk-variable
	test_sample = array(imputed[-n_lag:])
	# enumerate lead times
	lead_times = get_lead_times()
	for lead_time in lead_times:
		# make input/output data from series
		train_samples = supervised_for_lead_time(imputed, n_lag, lead_time)
		train_lead_times.append(train_samples)
	return train_lead_times, test_sample

# prepare training [var][lead time][sample] and test [chunk][var][sample]
def data_prep(chunks, n_lag, n_vars=39):
	lead_times = get_lead_times()
	train_data = [[list() for _ in range(len(lead_times))] for _ in range(n_vars)]
	test_data = [[list() for _ in range(n_vars)] for _ in range(len(chunks))]
	# enumerate targets for chunk
	for var in range(n_vars):
		# convert target number into column number
		col_ix = 3 + var
		# enumerate chunks to forecast
		for c_id in range(len(chunks)):
			rows = chunks[c_id]
			# prepare sequence of hours for the chunk
			hours = variable_to_series(rows, 2)
			# interpolate hours
			interpolate_hours(hours)
			# check for no data
			if not has_data(rows[:, col_ix]):
				continue
			# convert series into training data for each lead time
			train, test_sample = target_to_supervised(chunks, rows, hours, col_ix, n_lag)
			# store test sample for this var-chunk
			test_data[c_id][var] = test_sample
			if train is not None:
				# store samples per lead time
				for lead_time in range(len(lead_times)):
					# add all rows to the existing list of rows
					train_data[var][lead_time].extend(train[lead_time])
		# convert all rows for each var-lead time to a numpy array
		for lead_time in range(len(lead_times)):
			train_data[var][lead_time] = array(train_data[var][lead_time])
	return array(train_data), array(test_data)

# load dataset
train = loadtxt('naive_train.csv', delimiter=',')
test = loadtxt('naive_test.csv', delimiter=',')
# group data by chunks
train_chunks = to_chunks(train)
test_chunks = to_chunks(test)
# convert training data into supervised learning data
n_lag = 12
train_data, test_data = data_prep(train_chunks, n_lag)
print(train_data.shape, test_data.shape)


((39, 10), (207, 39))


In [22]:
# save train and test sets to file
save('supervised_train.npy', train_data)
save('supervised_test.npy', test_data)

结果是两个二进制文件，其中包含训练和测试数据集，我们可以在以下部分中加载它们，以训练和评估有关该问题的机器学习算法。

### 模型评估测试工具
在开始评估算法之前，我们需要测试工具的更多元素。

首先，我们需要能够在训练数据上拟合scikit学习模型。下面的fit_model（）函数将克隆模型配置并将其适合提供的训练数据。我们将需要为每个配置的模型配备许多（360）版本，因此将大量调用此功能。

In [23]:
# fit a single model
def fit_model(model, X, y):
	# clone the model configuration
	local_model = clone(model)
	# fit the model
	local_model.fit(X, y)
	return local_model

接下来，我们需要为每个变量和预测提前期组合拟合模型。

我们可以通过先按变量然后按交货时间枚举训练数据集来做到这一点。然后，我们可以拟合模型并将其存储在具有相同结构的列表列表中，特别是：[var] [time] [model]。

下面的fit_models（）函数实现了这一点。

In [24]:
# fit one model for each variable and each forecast lead time [var][time][model]
def fit_models(model, train):
	# prepare structure for saving models
	models = [[list() for _ in range(train.shape[1])] for _ in range(train.shape[0])]
	# enumerate vars
	for i in range(train.shape[0]):
		# enumerate lead times
		for j in range(train.shape[1]):
			# get data
			data = train[i, j]
			X, y = data[:, :-1], data[:, -1]
			# fit model
			local_model = fit_model(model, X, y)
			models[i][j].append(local_model)
	return models

拟合模型是较慢的部分，可以从并行化中受益，例如使用Joblib库。这留作扩展。

一旦模型拟合，就可以将其用于测试数据集的预测。

准备好的测试数据集首先按块组织，然后按目标变量组织。进行预测的速度很快，并且需要首先检查是否可以进行预测（我们有输入数据），如果可以，则对目标变量使用适当的模型。然后将使用这些提前期的每个直接模型来预测变量的10个预测提前期中的每一个。

下面的make_predictions（）函数实现了这一点，将模型列表列表和已加载的测试数据集作为参数，并返回结构为[chunks] [var] [time]的预测数组。

In [25]:
# return forecasts as [chunks][var][time]
def make_predictions(models, test):
	lead_times = get_lead_times()
	predictions = list()
	# enumerate chunks
	for i in range(test.shape[0]):
		# enumerate variables
		chunk_predictions = list()
		for j in range(test.shape[1]):
			# get the input pattern for this chunk and target
			pattern = test[i,j]
			# assume a nan forecast
			forecasts = array([nan for _ in range(len(lead_times))])
			# check we can make a forecast
			if has_data(pattern):
				pattern = pattern.reshape((1, len(pattern)))
				# forecast each lead time
				forecasts = list()
				for k in range(len(lead_times)):
					yhat = models[j][k][0].predict(pattern)
					forecasts.append(yhat[0])
				forecasts = array(forecasts)
			# save forecasts fore each lead time for this variable
			chunk_predictions.append(forecasts)
		# save forecasts for this chunk
		chunk_predictions = array(chunk_predictions)
		predictions.append(chunk_predictions)
	return array(predictions)

我们需要评估模型的列表。

我们可以定义一个通用的get_models（）函数，该函数负责定义映射到已配置的scikit-learn模型对象的模型名字典。

In [26]:
# prepare a list of ml models
def get_models(models=dict()):
	# ...
	return models

最后，我们需要一个函数来驱动模型评估过程。

给定模型字典，列举模型，首先将模型矩阵拟合到训练数据上，对测试数据集进行预测，评估预测，然后对结果进行汇总。

下面的evaluate_models（）函数实现了这一点。

In [27]:
# evaluate a suite of models
def evaluate_models(models, train, test, actual):
	for name, model in models.items():
		# fit models
		fits = fit_models(model, train)
		# make predictions
		predictions = make_predictions(fits, test)
		# evaluate forecast
		total_mae, _ = evaluate_forecasts(predictions, actual)
		# summarize forecast
		summarize_error(name, total_mae)

现在，我们拥有评估机器学习模型所需的一切。

### 评估线性算法
在本节中，我们将抽查一套线性机器学习算法。

线性算法是那些假设输出是输入变量的线性函数的算法。这非常类似于ARIMA等经典时间序列预测模型的假设。

抽查意味着对一组模型进行评估，以便大致了解可行的方法。我们对任何优于简单自回归模型AR（2）的模型都感兴趣，该模型实现了大约0.487的MAE误差。

我们将使用默认配置测试八种线性机器学习算法。特别：

- 线性回归
- 套索线性回归
- 岭回归
- 弹性净回归
- 胡伯回归
- 拉索·拉斯线性回归
- 被动攻击性回归
- 随机梯度下降回归

我们可以在get_models（）函数中定义这些模型。

In [28]:
# prepare a list of ml models
def get_models(models=dict()):
	# linear models
	models['lr'] = LinearRegression()
	models['lasso'] = Lasso()
	models['ridge'] = Ridge()
	models['en'] = ElasticNet()
	models['huber'] = HuberRegressor()
	models['llars'] = LassoLars()
	models['pa'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3)
	models['sgd'] = SGDRegressor(max_iter=1000, tol=1e-3)
	print('Defined %d models' % len(models))
	return models

下面列出了完整的代码示例。

In [30]:
# evaluate linear algorithms
from numpy import load
from numpy import loadtxt
from numpy import nan
from numpy import isnan
from numpy import count_nonzero
from numpy import unique
from numpy import array
from sklearn.base import clone
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import SGDRegressor
 
# split the dataset by 'chunkID', return a list of chunks
def to_chunks(values, chunk_ix=0):
	chunks = list()
	# get the unique chunk ids
	chunk_ids = unique(values[:, chunk_ix])
	# group rows by chunk id
	for chunk_id in chunk_ids:
		selection = values[:, chunk_ix] == chunk_id
		chunks.append(values[selection, :])
	return chunks
 
# return true if the array has any non-nan values
def has_data(data):
	return count_nonzero(isnan(data)) < len(data)
 
# return a list of relative forecast lead times
def get_lead_times():
	return [1, 2, 3, 4, 5, 10, 17, 24, 48, 72]
 
# fit a single model
def fit_model(model, X, y):
	# clone the model configuration
	local_model = clone(model)
	# fit the model
	local_model.fit(X, y)
	return local_model
 
# fit one model for each variable and each forecast lead time [var][time][model]
def fit_models(model, train):
	# prepare structure for saving models
	models = [[list() for _ in range(train.shape[1])] for _ in range(train.shape[0])]
	# enumerate vars
	for i in range(train.shape[0]):
		# enumerate lead times
		for j in range(train.shape[1]):
			# get data
			data = train[i, j]
			X, y = data[:, :-1], data[:, -1]
			# fit model
			local_model = fit_model(model, X, y)
			models[i][j].append(local_model)
	return models
 
# return forecasts as [chunks][var][time]
def make_predictions(models, test):
	lead_times = get_lead_times()
	predictions = list()
	# enumerate chunks
	for i in range(test.shape[0]):
		# enumerate variables
		chunk_predictions = list()
		for j in range(test.shape[1]):
			# get the input pattern for this chunk and target
			pattern = test[i,j]
			# assume a nan forecast
			forecasts = array([nan for _ in range(len(lead_times))])
			# check we can make a forecast
			if has_data(pattern):
				pattern = pattern.reshape((1, len(pattern)))
				# forecast each lead time
				forecasts = list()
				for k in range(len(lead_times)):
					yhat = models[j][k][0].predict(pattern)
					forecasts.append(yhat[0])
				forecasts = array(forecasts)
			# save forecasts for each lead time for this variable
			chunk_predictions.append(forecasts)
		# save forecasts for this chunk
		chunk_predictions = array(chunk_predictions)
		predictions.append(chunk_predictions)
	return array(predictions)
 
# convert the test dataset in chunks to [chunk][variable][time] format
def prepare_test_forecasts(test_chunks):
	predictions = list()
	# enumerate chunks to forecast
	for rows in test_chunks:
		# enumerate targets for chunk
		chunk_predictions = list()
		for j in range(3, rows.shape[1]):
			yhat = rows[:, j]
			chunk_predictions.append(yhat)
		chunk_predictions = array(chunk_predictions)
		predictions.append(chunk_predictions)
	return array(predictions)
 
# calculate the error between an actual and predicted value
def calculate_error(actual, predicted):
	# give the full actual value if predicted is nan
	if isnan(predicted):
		return abs(actual)
	# calculate abs difference
	return abs(actual - predicted)
 
# evaluate a forecast in the format [chunk][variable][time]
def evaluate_forecasts(predictions, testset):
	lead_times = get_lead_times()
	total_mae, times_mae = 0.0, [0.0 for _ in range(len(lead_times))]
	total_c, times_c = 0, [0 for _ in range(len(lead_times))]
	# enumerate test chunks
	for i in range(len(test_chunks)):
		# convert to forecasts
		actual = testset[i]
		predicted = predictions[i]
		# enumerate target variables
		for j in range(predicted.shape[0]):
			# enumerate lead times
			for k in range(len(lead_times)):
				# skip if actual in nan
				if isnan(actual[j, k]):
					continue
				# calculate error
				error = calculate_error(actual[j, k], predicted[j, k])
				# update statistics
				total_mae += error
				times_mae[k] += error
				total_c += 1
				times_c[k] += 1
	# normalize summed absolute errors
	total_mae /= total_c
	times_mae = [times_mae[i]/times_c[i] for i in range(len(times_mae))]
	return total_mae, times_mae
 
# summarize scores
def summarize_error(name, total_mae):
	print('%s: %.3f MAE' % (name, total_mae))
 
# prepare a list of ml models
def get_models(models=dict()):
	# linear models
	models['lr'] = LinearRegression()
	models['lasso'] = Lasso()
	models['ridge'] = Ridge()
	models['en'] = ElasticNet()
	models['huber'] = HuberRegressor()
	models['llars'] = LassoLars()
	models['pa'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3)
	models['sgd'] = SGDRegressor(max_iter=1000, tol=1e-3)
	print('Defined %d models' % len(models))
	return models
 
# evaluate a suite of models
def evaluate_models(models, train, test, actual):
	for name, model in models.items():
		# fit models
		fits = fit_models(model, train)
		# make predictions
		predictions = make_predictions(fits, test)
		# evaluate forecast
		total_mae, _ = evaluate_forecasts(predictions, actual)
		# summarize forecast
		summarize_error(name, total_mae)
 
# load supervised datasets
train = load('supervised_train.npy', allow_pickle=True)
test = load('supervised_test.npy', allow_pickle=True)
print(train.shape, test.shape)
# load test chunks for validation
testset = loadtxt('naive_test.csv', delimiter=',')
test_chunks = to_chunks(testset)
actual = prepare_test_forecasts(test_chunks)
# prepare list of models
models = get_models()
# evaluate models
evaluate_models(models, train, test, actual)

((39, 10), (207, 39))
Defined 8 models
en: 0.595 MAE
ridge: 0.454 MAE
huber: 0.434 MAE
sgd: 0.457 MAE
pa: 0.856 MAE
lr: 0.454 MAE
llars: 0.631 MAE
lasso: 0.624 MAE


运行示例将为每种评估算法打印MAE。

我们可以看到，与简单的AR模型相比，许多算法都具有技巧，MAE低于0.487。

Huber回归似乎表现最佳（使用默认配置），MAE为0.434。

这很有趣，因为Huber回归或带有Huber损失的鲁棒回归是一种对训练数据集中的异常值具有鲁棒性的方法。这可能表明其他方法在进行更多数据准备（例如标准化和/或离群值删除）时可能会表现更好。

### 非线性算法
我们可以使用相同的框架来评估一组非线性和集成的机器学习算法的性能。

特别：

#### 非线性算法

- k最近邻居
- 分类和回归树
- 多余的树
- 支持向量回归

#### 合奏算法

- Adaboost
- 袋装决策树
- 随机森林
- 多余的树木
- 梯度提升机

下面的get_models（）函数定义了这9个模型。

In [31]:
# prepare a list of ml models
def get_models(models=dict()):
	# non-linear models
	models['knn'] = KNeighborsRegressor(n_neighbors=7)
	models['cart'] = DecisionTreeRegressor()
	models['extra'] = ExtraTreeRegressor()
	models['svmr'] = SVR()
	# # ensemble models
	n_trees = 100
	models['ada'] = AdaBoostRegressor(n_estimators=n_trees)
	models['bag'] = BaggingRegressor(n_estimators=n_trees)
	models['rf'] = RandomForestRegressor(n_estimators=n_trees)
	models['et'] = ExtraTreesRegressor(n_estimators=n_trees)
	models['gbm'] = GradientBoostingRegressor(n_estimators=n_trees)
	print('Defined %d models' % len(models))
	return models

下面提供了完整的代码清单。

In [32]:
# spot check nonlinear algorithms
from numpy import load
from numpy import loadtxt
from numpy import nan
from numpy import isnan
from numpy import count_nonzero
from numpy import unique
from numpy import array
from sklearn.base import clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
 
# split the dataset by 'chunkID', return a list of chunks
def to_chunks(values, chunk_ix=0):
	chunks = list()
	# get the unique chunk ids
	chunk_ids = unique(values[:, chunk_ix])
	# group rows by chunk id
	for chunk_id in chunk_ids:
		selection = values[:, chunk_ix] == chunk_id
		chunks.append(values[selection, :])
	return chunks
 
# return true if the array has any non-nan values
def has_data(data):
	return count_nonzero(isnan(data)) < len(data)
 
# return a list of relative forecast lead times
def get_lead_times():
	return [1, 2, 3, 4, 5, 10, 17, 24, 48, 72]
 
# fit a single model
def fit_model(model, X, y):
	# clone the model configuration
	local_model = clone(model)
	# fit the model
	local_model.fit(X, y)
	return local_model
 
# fit one model for each variable and each forecast lead time [var][time][model]
def fit_models(model, train):
	# prepare structure for saving models
	models = [[list() for _ in range(train.shape[1])] for _ in range(train.shape[0])]
	# enumerate vars
	for i in range(train.shape[0]):
		# enumerate lead times
		for j in range(train.shape[1]):
			# get data
			data = train[i, j]
			X, y = data[:, :-1], data[:, -1]
			# fit model
			local_model = fit_model(model, X, y)
			models[i][j].append(local_model)
	return models
 
# return forecasts as [chunks][var][time]
def make_predictions(models, test):
	lead_times = get_lead_times()
	predictions = list()
	# enumerate chunks
	for i in range(test.shape[0]):
		# enumerate variables
		chunk_predictions = list()
		for j in range(test.shape[1]):
			# get the input pattern for this chunk and target
			pattern = test[i,j]
			# assume a nan forecast
			forecasts = array([nan for _ in range(len(lead_times))])
			# check we can make a forecast
			if has_data(pattern):
				pattern = pattern.reshape((1, len(pattern)))
				# forecast each lead time
				forecasts = list()
				for k in range(len(lead_times)):
					yhat = models[j][k][0].predict(pattern)
					forecasts.append(yhat[0])
				forecasts = array(forecasts)
			# save forecasts for each lead time for this variable
			chunk_predictions.append(forecasts)
		# save forecasts for this chunk
		chunk_predictions = array(chunk_predictions)
		predictions.append(chunk_predictions)
	return array(predictions)
 
# convert the test dataset in chunks to [chunk][variable][time] format
def prepare_test_forecasts(test_chunks):
	predictions = list()
	# enumerate chunks to forecast
	for rows in test_chunks:
		# enumerate targets for chunk
		chunk_predictions = list()
		for j in range(3, rows.shape[1]):
			yhat = rows[:, j]
			chunk_predictions.append(yhat)
		chunk_predictions = array(chunk_predictions)
		predictions.append(chunk_predictions)
	return array(predictions)
 
# calculate the error between an actual and predicted value
def calculate_error(actual, predicted):
	# give the full actual value if predicted is nan
	if isnan(predicted):
		return abs(actual)
	# calculate abs difference
	return abs(actual - predicted)
 
# evaluate a forecast in the format [chunk][variable][time]
def evaluate_forecasts(predictions, testset):
	lead_times = get_lead_times()
	total_mae, times_mae = 0.0, [0.0 for _ in range(len(lead_times))]
	total_c, times_c = 0, [0 for _ in range(len(lead_times))]
	# enumerate test chunks
	for i in range(len(test_chunks)):
		# convert to forecasts
		actual = testset[i]
		predicted = predictions[i]
		# enumerate target variables
		for j in range(predicted.shape[0]):
			# enumerate lead times
			for k in range(len(lead_times)):
				# skip if actual in nan
				if isnan(actual[j, k]):
					continue
				# calculate error
				error = calculate_error(actual[j, k], predicted[j, k])
				# update statistics
				total_mae += error
				times_mae[k] += error
				total_c += 1
				times_c[k] += 1
	# normalize summed absolute errors
	total_mae /= total_c
	times_mae = [times_mae[i]/times_c[i] for i in range(len(times_mae))]
	return total_mae, times_mae
 
# summarize scores
def summarize_error(name, total_mae):
	print('%s: %.3f MAE' % (name, total_mae))
 
# prepare a list of ml models
def get_models(models=dict()):
	# non-linear models
	models['knn'] = KNeighborsRegressor(n_neighbors=7)
	models['cart'] = DecisionTreeRegressor()
	models['extra'] = ExtraTreeRegressor()
	models['svmr'] = SVR()
	# # ensemble models
	n_trees = 100
	models['ada'] = AdaBoostRegressor(n_estimators=n_trees)
	models['bag'] = BaggingRegressor(n_estimators=n_trees)
	models['rf'] = RandomForestRegressor(n_estimators=n_trees)
	models['et'] = ExtraTreesRegressor(n_estimators=n_trees)
	models['gbm'] = GradientBoostingRegressor(n_estimators=n_trees)
	print('Defined %d models' % len(models))
	return models
 
# evaluate a suite of models
def evaluate_models(models, train, test, actual):
	for name, model in models.items():
		# fit models
		fits = fit_models(model, train)
		# make predictions
		predictions = make_predictions(fits, test)
		# evaluate forecast
		total_mae, _ = evaluate_forecasts(predictions, actual)
		# summarize forecast
		summarize_error(name, total_mae)
 
# load supervised datasets
train = load('supervised_train.npy', allow_pickle=True)
test = load('supervised_test.npy', allow_pickle=True)
print(train.shape, test.shape)
# load test chunks for validation
testset = loadtxt('naive_test.csv', delimiter=',')
test_chunks = to_chunks(testset)
actual = prepare_test_forecasts(test_chunks)
# prepare list of models
models = get_models()
# evaluate models
evaluate_models(models, train, test, actual)

((39, 10), (207, 39))
Defined 9 models
knn: 0.484 MAE
extra: 0.629 MAE
gbm: 0.450 MAE
cart: 0.630 MAE


KeyboardInterrupt: 

通过运行示例，我们可以看到许多算法与自动回归算法的基线相比表现良好，尽管在上一节中没有一种算法和Huber回归一样好。

支持向量回归和梯度提升机可能都值得进一步研究，以分别达到0.437和0.450的MAE。

### 调滞大小
在以前的抽查实验中，滞后观察的次数被任意固定为12。

我们可以改变滞后观测值的数量，并评估对MAE的影响。某些算法可能需要更多或更少的先验观察结果，但总体趋势可能会贯穿算法。

使用一系列不同数量的滞后观测值准备监督学习数据集，并在每个值上拟合和评估HuberRegressor。

我尝试了以下滞后观察：

[1, 3, 6, 12, 24, 36, 48]

结果如下：   
1:	0.451   
3:	0.445   
6:	0.441   
12:	0.434   
24:	0.423   
36:	0.422   
48:	0.439   


我们可以看到总体MAE随着滞后观测次数的增加而下降的总体趋势，至少到误差开始再次上升的时候。

结果表明，至少对于HuberRegressor算法而言，36个滞后观测值可能是实现0.422的MAE的良好配置。


### 扩展
本节列出了一些扩展探索的想法。

- 数据准备。探索简单的数据准备（例如标准化或统计异常值消除）是否可以改善模型性能。
- 工程特性。探索工程特征（例如，一天中的预期小时中位数）是否可以改善模型性能
- 气象变量。探索将滞后气象变量添加到模型中是否可以提高性能。
- 跨站点模型。探索组合相同类型的目标变量并在站点之间重复使用模型是否可以提高性能。
- 算法调整。探索调整一些性能更好的算法的超参数是否可以提高性能。

### 摘要
在本教程中，我们发现了如何开发机器学习模型以对空气污染数据进行多步时间序列预测。

具体来说，我们了解到：

- 如何估算缺失值并转换时间序列数据，以便可以通过监督学习算法对其进行建模。
- 如何开发和评估一套用于多步时间序列预测的线性算法。
- 如何开发和评估一套用于多步时间序列预测的非线性算法。